# more sample about implement fro advanced landing from multi data-source

## import library

In [1]:
import dask.dataframe as dd
import pandas as pd
from data_lib import landing_dsm as dsm

/usr/local/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## declear data-source

In [2]:
# Database connection parameters
db_params = [{
    'appliction':'app01',
    'source':'db1',
    'dbname': 'chinook',
    'user': 'admin',
    'password': '1234',
    'host': 'postgresql-dev',
    'port': '5432'
    },
    {
    'appliction':'app01',
    'source':'db2',
    'dbname': 'northwind',
    'user': 'admin',
    'password': '1234',
    'host': 'postgresql-dev',
    'port': '5432'
    }
]

## list table from data-source and landing some table

In [3]:
tables = []
for db in db_params:
    
    db_name = db["dbname"]
    table = dsm.list_table(db_params=db)

    print(f"database {db_name} has table : {len(table)} table")
    print(table)

database chinook has table : 11 table
['artist', 'album', 'employee', 'customer', 'invoice', 'invoice_line', 'track', 'playlist', 'playlist_track', 'genre', 'media_type']
database northwind has table : 14 table
['territories', 'order_details', 'employee_territories', 'us_states', 'customers', 'orders', 'employees', 'shippers', 'products', 'categories', 'suppliers', 'region', 'customer_demographics', 'customer_customer_demo']


In [5]:
dsm.landing_func(db_params[0],["customer"])
dsm.landing_func(db_params[1],["customers"])

done
done


In [6]:
dataPath1 = "/landing_zone/app01/db1/chinook/customer.parquet"
dataPath2 = "/landing_zone/app01/db2/northwind/customers.parquet"

In [9]:
ddf1 = dsm.get_parquet(dataPath1)
ddf1.head()

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [10]:
ddf2 = dsm.get_parquet(dataPath2)
ddf2.head()

,customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67


## staging zone

In [11]:
ddf3 = ddf1[["customer_id","company"]]
ddf3["contact_name"] = ddf1["first_name"]+" "+ddf1["last_name"]
ddf3.compute().info()
ddf3.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   customer_id   59 non-null     int32 
 1   company       10 non-null     object
 2   contact_name  59 non-null     object
dtypes: int32(1), object(2)
memory usage: 1.3+ KB


,customer_id,company,contact_name
0,1,Embraer - Empresa Brasileira de Aeronáutica S.A.,Luís Gonçalves
1,2,None,Leonie Köhler
2,3,None,François Tremblay
3,4,None,Bjørn Hansen
4,5,JetBrains s.r.o.,František Wichterlová


In [12]:
ddf4 = ddf2[["customer_id","company_name","contact_name"]]
ddf4.compute().info()
ddf4.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   customer_id   91 non-null     object
 1   company_name  91 non-null     object
 2   contact_name  91 non-null     object
dtypes: object(3)
memory usage: 2.3+ KB


,customer_id,company_name,contact_name
0,ALFKI,Alfreds Futterkiste,Maria Anders
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo
2,ANTON,Antonio Moreno Taquería,Antonio Moreno
3,AROUT,Around the Horn,Thomas Hardy
4,BERGS,Berglunds snabbköp,Christina Berglund


In [13]:
integate_ddf = dd.concat([ddf3,ddf4])
integate_ddf.compute().info()
integate_ddf.head()

<class 'pandas.core.frame.DataFrame'>
Index: 150 entries, 0 to 90
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   customer_id   150 non-null    object
 1   company       10 non-null     object
 2   contact_name  150 non-null    object
 3   company_name  91 non-null     object
dtypes: object(4)
memory usage: 5.9+ KB


,customer_id,company,contact_name,company_name
0,1,Embraer - Empresa Brasileira de Aeronáutica S.A.,Luís Gonçalves,NaN
1,2,None,Leonie Köhler,NaN
2,3,None,François Tremblay,NaN
3,4,None,Bjørn Hansen,NaN
4,5,JetBrains s.r.o.,František Wichterlová,NaN
